In [1]:
!wget https://huggingface.co/garyw/clinical-embeddings-100d-w2v-cr/resolve/main/w2v_OA_CR_100d.bin
!wget https://huggingface.co/garyw/clinical-embeddings-100d-w2v-cr/resolve/main/w2v_OA_CR_100d.bin.wv.vectors.npy
!wget https://huggingface.co/garyw/clinical-embeddings-100d-w2v-cr/resolve/main/w2v_OA_CR_100d.bin.trainables.syn1neg.npy
!pip install fastparquet

--2024-06-04 12:17:21--  https://huggingface.co/garyw/clinical-embeddings-100d-w2v-cr/resolve/main/w2v_OA_CR_100d.bin
Resolving huggingface.co (huggingface.co)... 18.172.134.124, 18.172.134.88, 18.172.134.4, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/e8/72/e872abef7b2278b5cf6a4f41d1455f5d309cdbd6310b2da0a6e9aecc840217e7/7f920b29a2c3861a475f296d397ff679528b064dbb51debf33568ae5f4f7e088?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27w2v_OA_CR_100d.bin%3B+filename%3D%22w2v_OA_CR_100d.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1717762641&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNzc2MjY0MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9lOC83Mi9lODcyYWJlZjdiMjI3OGI1Y2Y2YTRmNDFkMTQ1NWY1ZDMwOWNkYmQ2MzEwYjJkYTBhNmU5YWVjYzg0MDIxN2U3LzdmOTIwYjI5YTJjM

In [46]:
from gensim.models import FastText, Word2Vec, KeyedVectors
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from nltk.stem import WordNetLemmatizer
import string
import nltk
from torch.utils.data import Dataset,DataLoader, random_split
import re
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
model = Word2Vec.load('w2v_OA_CR_100d.bin')
print(model.wv.get_vector('lymphangioleiomyomatosis'))

[-0.6044092   0.4017609  -0.717026   -0.2701869  -0.23817156 -0.34242344
 -0.15332928 -0.11580862 -0.36377856 -0.10749034  0.4498769   0.7072215
 -0.4689228  -0.48433578  1.050483    0.71655744 -0.6439546  -0.44996452
 -0.11657333  0.09194979  0.35634688  0.4884644   0.28647774 -0.496315
 -0.7018925  -0.25540805  0.14061369 -0.8933765  -0.52663106  0.40501425
  0.17843099  0.21091795 -0.231396   -0.18487869  0.38206643  0.09275728
 -0.4573524   0.7668036   0.17861798  0.31376782 -0.49365893 -0.5377006
  0.27592292 -0.04515044 -0.23638678 -0.5425362  -0.46474802  0.10646814
 -0.17016436 -0.07704978  0.17462458  0.27287003  0.5647319   0.1521702
  0.1517999   0.2716373   0.21111147 -0.11035519  0.19445771  0.14426446
  0.7005133  -0.71069545  0.01735174  0.30701727 -0.54055333 -0.0802884
 -0.1946127  -0.4616384  -0.4518422  -0.00218993 -0.3471012   0.60170263
 -0.47892392 -0.04716599  0.07105272  0.35743713 -0.4405513  -0.53299844
 -0.21108003  0.10141873  0.25599658 -0.22999842  0.21882

In [4]:
test_data = pd.read_parquet('test-00000-of-00001-47685aa42db61e77.parquet', engine='fastparquet')
train_data = pd.read_parquet('train-00000-of-00001-210cfe9263b99806.parquet', engine='fastparquet')
valid_data = pd.read_parquet('valid-00000-of-00001-cc552de6d1a6fa4b.parquet', engine='fastparquet')
test_data

,id,query,answer,choices,gold
0,MedNLI0,\nTASK: Please classify the relationship betwe...,entailment,"[entailment, contradiction, neutral]",0
1,MedNLI1,\nTASK: Please classify the relationship betwe...,contradiction,"[entailment, contradiction, neutral]",1
2,MedNLI2,\nTASK: Please classify the relationship betwe...,neutral,"[entailment, contradiction, neutral]",2
3,MedNLI3,\nTASK: Please classify the relationship betwe...,entailment,"[entailment, contradiction, neutral]",0
4,MedNLI4,\nTASK: Please classify the relationship betwe...,contradiction,"[entailment, contradiction, neutral]",1
...,...,...,...,...,...
1417,MedNLI1417,\nTASK: Please classify the relationship betwe...,contradiction,"[entailment, contradiction, neutral]",1
1418,MedNLI1418,\nTASK: Please classify the relationship betwe...,neutral,"[entailment, contradiction, neutral]",2
1419,MedNLI1419,\nTASK: Please classify the relationship betwe...,entailment,"[entailment, contradiction, neutral]",0
1420,MedNLI1420,\nTASK: Please classify the relationship betwe...,contradiction,"[entailment, contradiction, neutral]",1


In [5]:
def find_pre_and_hyp(query):
    start_pre = query.find("[PRE]") + len("[PRE]")
    end_pre = query.find("[HYP]")
    start_hyp = query.find("[HYP]") + len("[HYP]")
    end_hyp = query.find("OUTPUT:")
    premise = query[start_pre:end_pre].strip()
    hypothesis = query[start_hyp:end_hyp].strip()

    return premise,hypothesis

In [6]:
premise_texts_tokens = []
hypothesis_texts_tokens = []
lemmatizer = WordNetLemmatizer()
for query,answer in zip(train_data['query'],train_data['answer']):
    premise,hypothesis = find_pre_and_hyp(query)

    premise = premise.replace('-','')
    hypothesis = hypothesis.replace('-','')
    #premise = re.sub
    #hypothesis =

    tokens_premise = nltk.word_tokenize(premise)
    tokens_hypothesis = nltk.word_tokenize(hypothesis)

    tokens_premise = [lemmatizer.lemmatize(token.lower()) for token in tokens_premise if token not in string.punctuation]
    tokens_hypothesis = [lemmatizer.lemmatize(token.lower()) for token in tokens_hypothesis if token not in string.punctuation]

    premise_texts_tokens.append(tokens_premise)
    hypothesis_texts_tokens.append(tokens_hypothesis)

In [21]:
def get_vectors_of_given_wordd(word):
    try:
       return model.wv.get_vector(word)
    except:
       #print(word)
       return None

In [19]:
print(get_vectors_of_given_wordd('Number'))

Number
None


In [22]:
x_premise_arr = []
max_len = 0
for text in premise_texts_tokens:
    x = np.empty((0,100))
    for word in text:
        vectors = get_vectors_of_given_wordd(word)
        if not vectors is None:
            vectors_np = np.array(vectors)
            x = np.vstack([x, vectors_np])
    if max_len < x.shape[0]:
         max_len = x.shape[0]
    x_premise_arr.append(x)

In [23]:
x_hypothesis_arr = []
for text in hypothesis_texts_tokens:
    x = np.empty((0,100))
    for word in text:
        vectors = get_vectors_of_given_wordd(word)
        if not vectors is None:
            vectors_np = np.array(vectors)
            x = np.vstack([x, vectors_np])
    if max_len < x.shape[0]:
         max_len = x.shape[0]
    x_hypothesis_arr.append(x)

In [32]:
y_arr = []
for query,answer in zip(train_data['query'],train_data['answer']):
    if answer == 'entailment':
       y = [1,0,0]
    elif answer == 'neutral':
       y = [0,1,0]
    elif answer == 'contradiction':
       y = [0,0,1]
    else:
        print('should not get here')
    y_arr.append(y)

    #premise,hypothesis = find_pre_and_hyp(query)

In [16]:
for x_premise in x_premise_arr:
    if x_premise.shape[0] == 0:
       print(x_premise.shape)

In [17]:
for x_hypothesis in x_hypothesis_arr:
    if x_hypothesis.shape[0] == 0:
       print(x_hypothesis.shape)

In [34]:
print(len(x_hypothesis_arr))
print(len(x_premise_arr))

11232
11232
